# General Imports

**Importing all libraries**

In [1]:
import numpy as np
import pandas as pd
import warnings

import plotly.graph_objects as go
from bayes_opt import BayesianOptimization
import statsmodels.api as sm

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance


Disable warnings

In [2]:
# Disable all warnings
warnings.filterwarnings("ignore")

# Enable warnings again
## warnings.filterwarnings("default")

Generate train and test data

In [3]:
# Read Data
df = pd.read_csv("data_finish_prep.csv")

# Select only the usefull part of the df
df.drop(columns=['verschil_Lengte',
               'verschil_6 MWT', 'verschil_TUG',
               'verschil_BMI', 'verschil_Conditie',
               'verschil_Gewicht', 'verschil_Knijpkracht'], inplace=True)

# Drop NaN values
df.dropna(inplace=True)

# Define X and y
X = df.drop(columns=['verschil_Lenigheid'])
y = df['verschil_Lenigheid']

# Split data into train & test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# MLR

**feature selection**

In [5]:
# Fit the Full Model
X =  sm.add_constant(X_train)  # Add constant term for the intercept
model = sm.OLS(y_train, X_train).fit()  # Fit the MLR model
best_model = model  # Initialize the best model
selected_features = X_train.columns.tolist()
best_features = X_train.columns.tolist()
best_features_rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

# Iteratively remove one feature at a time based on p-values
while True:
    # Compute p-values
    p_values = model.pvalues[1:]  # Exclude the constant term

    # Identify Insignificant Features
    insignificant_feature = p_values.idxmax()
    max_p_value = p_values.max()

    # Remove Insignificant Feature
    X = X.drop(insignificant_feature, axis=1)
    selected_features.remove(insignificant_feature)

    # When there a no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    model = sm.OLS(y_train, X).fit()
    
    # Evaluate Model Performance
    rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error').mean())

    # Check if the model performs better without the insignificant features
    if rmse <= best_features_rmse:
        best_features_rmse = rmse
        best_features = selected_features.copy()
        best_model = model

# Save results
best_model_mlr = LinearRegression().fit(X_train[best_features], y_train)
best_features_mlr = best_features

# Show results
print("Best features:", best_features)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features)))

Best features: ['GeboorteRelatief', 'ALandbouwBosbouwEnVisserij', 'AfstandTotSchool', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotSportterrein', 'AfstandTotMuseum', 'ErvarenGezondheidGoedZeerGoed', 'ErnstigOvergewichtObesitas', 'UrenMantelzorgPerWeek']
Dropped features: ['AfstandTotBioscoop', 'AfstandTotRestaurant', 'AfstandTotSemiOpenbaarGroenTotaal', 'AfstandTotSauna', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotOpenDroogNatTerrein', 'NederlandseAntillenEnAruba', 'ZwaarBelasteMantelzorgers', 'OppervlakteWater', 'BeperkingInZien', 'PercentageOnbewoond', 'AfstandTotHuisartsenpost', 'AfstandTotBos', 'GemiddeldeWoningwaarde', 'AfstandTotWarenhuis', 'AfstandTotBegraafplaats', 'AfstandTotZonnebank', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotOpenNatTerreinTotaal', 'AfstandTotBibliotheek', 'AfstandTotCafeED', 'Ondergewicht', 'MatigOvergewicht', 'VoldoetAanBeweegrichtlijn', 'InBezitWoningcorporatie', 'AfstandTotZwembad', 'AfstandTotPoppodium', 'AfstandTotBrandweerkazerne', 'A

*Er zijn geen hyperparameters om te optimaliseren.*

# Support Vector Machines

**Hyperparameter optimalisatie**

In [6]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    classifier = model(**params)
    
    # Calculating accuracy based on cross-validation
    scores = cross_val_score(classifier, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
    return scores.mean()

params_ranges = {
    'C': (0.1, 10),
    'kernel_int': (1, 4),
    'gamma': (0.001, 0.1)
}

kernel_mapping = {
    1: 'linear',
    2: 'rbf',
    3: 'poly',
    4: 'sigmoid'
}

# Example usage with SVM
model = SVR
model_bo = BayesianOptimization(f=lambda C, kernel_int, gamma:
                                    bo_params_generic(model, {
                                        'C': C,
                                        'kernel': kernel_mapping[int(kernel_int)],
                                        'gamma': gamma
                                    }, X_train, y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_svm = model(
    C=params['C'],
    kernel=kernel_mapping[int(params['kernel_int'])],
    gamma=params['gamma']
)

# Fit the model
best_model_svm.fit(X_train, y_train)


|   iter    |  target   |     C     |   gamma   | kernel... |
-------------------------------------------------------------


| 1         | -40.25    | 2.161     | 0.05162   | 1.883     |
| 2         | -40.22    | 6.319     | 0.0546    | 1.304     |
| 3         | -40.27    | 8.27      | 0.03721   | 3.711     |
| 4         | -40.3     | 6.465     | 0.01071   | 2.532     |
| 5         | -40.2     | 3.104     | 0.01005   | 3.305     |
| 6         | -40.29    | 5.734     | 0.03661   | 2.975     |
| 7         | -40.25    | 8.413     | 0.04796   | 1.258     |
| 8         | -40.33    | 4.812     | 0.06648   | 3.684     |
| 9         | -40.33    | 5.268     | 0.06415   | 3.652     |
| 10        | -40.25    | 3.311     | 0.05962   | 1.344     |
| 11        | -40.55    | 2.305     | 0.001     | 3.368     |
| 12        | -40.29    | 3.342     | 0.01884   | 3.291     |
| 13        | -40.25    | 3.093     | 0.01565   | 3.278     |
| 14        | -40.22    | 5.488     | 0.003116  | 2.706     |
| 15        | -40.32    | 9.352     | 0.04939   | 3.02      |
| 16        | -40.25    | 3.34      | 0.02372   | 1.979     |
| 17    

SVR(C=0.8389103323874555, gamma=0.0054424540530956115)

**Feature Selection**

In [7]:
# Get the accuracy score of the model with all features
best_accuracy = cross_val_score(best_model_svm, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()
best_model = best_model_svm
selected_features = X_train.columns
best_features = selected_features

while len(selected_features) > 1:
    try:
        # Retrieve the coefficients from the SVM model
        coefficients = best_model_svm.coef_[0]

        # Sort features based on their coefficient magnitudes
        sorted_indices = np.argsort(np.abs(coefficients))
        selected_features = [X_train.columns[i] for i in sorted_indices[::-1]]

        # Remove the least important feature
        selected_features = selected_features[:-1]
        
        # Refit the linear SVM classifier
        best_model_svm_classification.fit(X_train[selected_features], y_train)
        
        # Evaluate Model Performance
        accuracy = cross_val_score(best_model_svm, X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

        # Check if the model performs better without the insignificant features
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_features = selected_features.copy()
            best_model = best_model_svm
    except:
        print("Scince the kernal is not linear, this feature selection is not possible") 
        break

# Save results
best_model_svm = best_model
best_features_svm = best_features

# Show results
print("Best features:", best_features_svm)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_svm)))

Scince the kernal is not linear, this feature selection is not possible
Best features: Index(['NederlandseAntillenEnAruba', 'GeboorteRelatief', 'SterfteRelatief',
       'GemiddeldeWoningwaarde', 'PercentageOnbewoond',
       'InBezitWoningcorporatie', 'InBezitOverigeVerhuurders',
       'EigendomOnbekend', 'BouwjaarVanaf2000', 'kPersonenMetLaagsteInkomen',
       'ALandbouwBosbouwEnVisserij', 'KLFinancieleDienstenOnroerendGoed',
       'PersonenautoSOverigeBrandstof', 'PersonenautoSPerHuishouden',
       'PersonenautoSNaarOppervlakte', 'AfstandTotHuisartsenpraktijk',
       'AfstandTotKinderdagverblijf', 'AfstandTotSchool', 'OppervlakteWater',
       'TotaalDiefstalUitWoningSchuurED',
       'VernielingMisdrijfTegenOpenbareOrde', 'GeweldsEnSeksueleMisdrijven',
       'AfstandTotHuisartsenpost', 'AfstandTotApotheek',
       'AfstandTotConsultatiebureau', 'AfstandTotOvDagelLevensmiddelen',
       'AfstandTotWarenhuis', 'AfstandTotCafeED', 'AfstandTotCafetariaED',
       'AfstandTotResta

# Random Forest

**Feature selection**

In [8]:
# Get the accuracy score of the model with all the features
rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train, y_train)
best_rf = rf
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Make DataFrame of feature impotances
    result = permutation_importance(
            rf, X_train[selected_features], y_train, n_repeats=10, random_state=42, n_jobs=-1
        )
    df_importances = pd.DataFrame({'feature': X_train[selected_features].columns, 'importance': result.importances_mean, 'std': result.importances_std})
    df_importances = df_importances.sort_values('importance')

    # Remove least important feature
    selected_features.remove(df_importances['feature'].iloc[0])

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = rf

# Save results
best_model_rf = best_model
best_features_rf = best_features

# Show results
print("Best features:", best_features_rf)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_rf)))

Best features: ['AfstandTotHotelED', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotKunstijsbaan']
Dropped features: ['AfstandTotMuseum', 'AfstandTotBioscoop', 'AfstandTotRestaurant', 'AfstandTotSemiOpenbaarGroenTotaal', 'AfstandTotSauna', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotOpenDroogNatTerrein', 'NederlandseAntillenEnAruba', 'ZwaarBelasteMantelzorgers', 'OppervlakteWater', 'BeperkingInZien', 'AfstandTotSportterrein', 'PercentageOnbewoond', 'AfstandTotHuisartsenpost', 'AfstandTotBos', 'GemiddeldeWoningwaarde', 'AfstandTotWarenhuis', 'AfstandTotBegraafplaats', 'AfstandTotZonnebank', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotOpenNatTerreinTotaal', 'AfstandTotBibliotheek', 'AfstandTotCafeED', 'Ondergewicht', 'MatigOvergewicht', 'VoldoetAanBeweegrichtlijn', 'InBezitWoningcorporatie', 'AfstandTotZwembad', 'AfstandTotPoppodium', 'AfstandTotBrandweerkazerne', 'AfstandTotCafetariaED', 'SterfteRelatief', 'AfstandTotBelangrijkOverstapstation', 'ALandbouwBosbouwEnVisserij', 

**Hyperparameter optimalisatie**

In [9]:
def bo_params_rf(min_samples_split,max_depth,max_features,n_estimators):
    
    params = {
        'min_samples_split': round(min_samples_split),
        'max_depth': round(max_depth),
        'max_features': max_features,
        'n_estimators':round(n_estimators)
    }    
        
    clf = RandomForestRegressor(min_samples_split=int(params['min_samples_split']), 
                                 max_depth=int(params['max_depth']), 
                                 max_features=params['max_features'],
                                 n_estimators=int(params['n_estimators']), 
                                 bootstrap=False,
                                 n_jobs = -1)
    return cross_val_score(clf, X_train[best_features_rf], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

# Create Bayesian Optimization
rf_bo = BayesianOptimization(f=bo_params_rf, pbounds={
    'n_estimators': (100, 1000),
    'max_depth': (1, 20),
    'max_features': (0.1, 1),
    'min_samples_split': (2, 12)
})

# Bayesian optimization
results = rf_bo.maximize(n_iter=40, init_points=10)

# Getting best hyperparameters from bayesian optimization
params = rf_bo.max['params']
params['n_estimators']= round(params['n_estimators'])
params['min_samples_split']= round(params['min_samples_split'])
params['max_depth']= round(params['max_depth'])

# Making random forest model with the best hyperparameters
# criterion entropy is after the hyperparameter optimization since it's more accurate but increases training time
best_model_rf_classification = RandomForestRegressor(min_samples_split=params['min_samples_split'],
                                                max_depth=params['max_depth'],
                                                max_features=params['max_features'],
                                                n_estimators=params['n_estimators'],
                                                bootstrap=False)

best_model_rf_classification.fit(X_train[best_features_rf], y_train)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | -40.1     | 3.129     | 0.8523    | 6.341     | 727.2     |
| 2         | -40.08    | 11.8      | 0.5172    | 5.467     | 709.6     |
| 3         | -40.01    | 2.254     | 0.1926    | 11.71     | 323.4     |
| 4         | -40.15    | 12.09     | 0.8239    | 3.298     | 516.6     |
| 5         | -39.93    | 3.751     | 0.1231    | 8.699     | 680.1     |
| 6         | -40.14    | 10.01     | 0.8345    | 8.751     | 233.8     |
| 7         | -40.08    | 17.97     | 0.4666    | 8.429     | 227.8     |
| 8         | -39.95    | 3.404     | 0.4272    | 11.08     | 415.6     |
| 9         | -40.14    | 10.65     | 0.9097    | 3.581     | 309.8     |
| 10        | -40.15    | 8.027     | 0.9791    | 7.741     | 850.9     |
| 11        | -39.94    | 3.198     | 0.6492    | 9.817     | 681.2     |
| 12        | -39.95    | 1.118     | 

RandomForestRegressor(bootstrap=False, max_depth=1, max_features=0.1,
                      min_samples_split=12, n_estimators=677)

# neural network

**Feature selection**

In [10]:
# Get the accuracy score of the model with all the features
mlp = MLPRegressor(random_state=0).fit(X_train, y_train)
best_model = mlp
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(MLPRegressor(random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Retrieve the learned weights
    weights = mlp.coefs_[0]  # Weights connecting input features to the first hidden layer

    # Normalize the weights
    normalized_weights = np.abs(weights) / np.sum(np.abs(weights), axis=0)

    # Calculate feature importance
    feature_importance = np.mean(normalized_weights, axis=1)

    # Sort features based on their importance
    sorted_indices = np.argsort(feature_importance)
    selected_features = [selected_features[i] for i in sorted_indices[:-1]]

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    mlp = MLPRegressor(random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(MLPRegressor(random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = mlp

# Save results
best_model_nn = best_model
best_features_nn = best_features

# Show results
print("Best features:", best_features_nn)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_nn)))

Best features: ['AfstandTotBelangrijkOverstapstation', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotCafeED', 'NederlandseAntillenEnAruba', 'PersonenautoSPerHuishouden', 'AfstandTotPoppodium', 'SterfteRelatief', 'AfstandTotHuisartsenpraktijk', 'ALandbouwBosbouwEnVisserij', 'Ondergewicht', 'AfstandTotBegraafplaats', 'TotaalDiefstalUitWoningSchuurED', 'ErnstigOvergewichtObesitas', 'InBezitOverigeVerhuurders']
Dropped features: ['AfstandTotMuseum', 'AfstandTotBioscoop', 'AfstandTotRestaurant', 'AfstandTotSemiOpenbaarGroenTotaal', 'AfstandTotSauna', 'AfstandTotOpenDroogNatTerrein', 'ZwaarBelasteMantelzorgers', 'OppervlakteWater', 'BeperkingInZien', 'AfstandTotSportterrein', 'AfstandTotOpenNatNatuurlijkTerrein', 'PercentageOnbewoond', 'AfstandTotHuisartsenpost', 'AfstandTotBos', 'GemiddeldeWoningwaarde', 'AfstandTotWarenhuis', 'AfstandTotZonnebank', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotOpenNatTerreinTotaal', 'AfstandTotBibliotheek', 'MatigOvergewicht', 'VoldoetAanBeweegrichtl

**Hyperparameter optimalisatie**

In [11]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Calculating rmse based on
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return scores.mean()

params_ranges = {
    'hidden_layer_sizes': (10, 100),
    'alpha': (0.0001, 0.1),
    'learning_rate_init': (0.001, 0.1),
    'max_iter': (100, 1000)
}

# Example usage with Neural Network
model = MLPRegressor
model_bo = BayesianOptimization(f=lambda hidden_layer_sizes, alpha, learning_rate_init, max_iter:
                                    bo_params_generic(model, {
                                        'hidden_layer_sizes': (int(round(hidden_layer_sizes)),),
                                        'alpha': alpha,
                                        'learning_rate_init': learning_rate_init,
                                        'max_iter': int(round(max_iter))
                                    }, X_train[best_features_nn], y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_nn = model(
    hidden_layer_sizes=(int(round(params['hidden_layer_sizes'])),),
    alpha=params['alpha'],
    learning_rate_init=params['learning_rate_init'],
    max_iter=int(round(params['max_iter']))
)

# Fit the model
best_model_nn.fit(X_train[best_features_nn], y_train)

|   iter    |  target   |   alpha   | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------
| 1         | -6.316    | 0.0517    | 63.37     | 0.05341   | 241.5     |
| 2         | -6.314    | 0.07993   | 19.1      | 0.005022  | 822.7     |
| 3         | -6.34     | 0.04186   | 53.56     | 0.05991   | 928.1     |
| 4         | -6.325    | 0.05354   | 60.14     | 0.05987   | 134.9     |
| 5         | -6.33     | 0.006379  | 89.96     | 0.07883   | 828.4     |
| 6         | -6.329    | 0.05151   | 82.39     | 0.03464   | 594.8     |
| 7         | -6.322    | 0.06436   | 53.29     | 0.06099   | 722.8     |
| 8         | -6.335    | 0.02121   | 52.33     | 0.0947    | 758.2     |
| 9         | -6.323    | 0.09895   | 94.48     | 0.02001   | 949.2     |
| 10        | -6.342    | 0.08639   | 30.33     | 0.04893   | 383.7     |
| 11        | -6.332    | 0.02184   | 63.95     | 0.05781   | 242.9     |
| 12        | -6.326    | 0.007006  | 

MLPRegressor(alpha=0.07607286962003253, hidden_layer_sizes=(43,),
             learning_rate_init=0.07145203076172235, max_iter=188)

# plotting the bar chart

**using train dataset**

In [12]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_val = []

for model in regression_models:
    try:
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=10)
        y_true = y_train
    except: # cv can't be higher then n_splits in the dt
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=5)
        y_true = y_train

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_val.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_val

[{'model': 'Multiple Linear Regression',
  'mae': 4.264611503438242,
  'mse': 40.11965623451727,
  'rmse': 6.334007912413536,
  'r2': 0.008052087655813667,
  'adj_r2': -0.06669102737550281},
 {'model': 'Random Forest',
  'mae': 4.322654777866058,
  'mse': 40.11135702458488,
  'rmse': 6.333352747525191,
  'r2': 0.008257283431131857,
  'adj_r2': -0.06647037013164603},
 {'model': 'Neural Network',
  'mae': 4.287658798082273,
  'mse': 40.454843548256825,
  'rmse': 6.36041221527794,
  'r2': -0.0002353302164797011,
  'adj_r2': -0.07560290084635435},
 {'model': 'Support Vector Machine',
  'mae': 4.224581990775344,
  'mse': 40.132381445923215,
  'rmse': 6.335012347732498,
  'r2': 0.007737460162136278,
  'adj_r2': -0.06702936200002774}]

In [13]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_val]
metric_labels = list(regression_stats_val[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_val] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

**Evaluating on test data**

In [14]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_test = []

for model in regression_models:
    # Make predictions on the test set
    y_pred = model['model'].predict(X_test[model['features']])

    # Defining Actual values
    y_true = y_test.copy()

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_test.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_test

[{'model': 'Multiple Linear Regression',
  'mae': 3.938800607982198,
  'mse': 33.87969414469903,
  'rmse': 5.8206266110015195,
  'r2': -0.004265664212496079,
  'adj_r2': -0.39481342251735563},
 {'model': 'Random Forest',
  'mae': 3.9341030694094976,
  'mse': 33.660782863811036,
  'rmse': 5.801791349558431,
  'r2': 0.002223328337602859,
  'adj_r2': -0.3858009328644405},
 {'model': 'Neural Network',
  'mae': 3.888797858771391,
  'mse': 33.14580861110289,
  'rmse': 5.75723966941649,
  'r2': 0.017488252446405617,
  'adj_r2': -0.36459964937999234},
 {'model': 'Support Vector Machine',
  'mae': 3.8638537025283712,
  'mse': 33.27785145409668,
  'rmse': 5.768695819168894,
  'r2': 0.013574223799709295,
  'adj_r2': -0.3700358002781814}]

In [15]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_test]
metric_labels = list(regression_stats_test[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_test] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

Naive bayes

In [16]:
# Defining Actual values
y_true = y_test.copy()

# Defining Predicted values
y_pred = np.full_like(y_test, y_train.mean())

# Mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Mean Squared Error
mse = mean_squared_error(y_true, y_pred)
print("Mean Squared Error:", mse)

# Root Mean Squared Error
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

# R2 Score
r2 = r2_score(y_true, y_pred)
print("R2 Score:", r2)

# Adjusted R2 Score
n = len(y_true)  # Number of samples
p = X_test.shape[1]  # Number of predictors (features) in X
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)
print("Adjusted R2 Score:", adj_r2)

Mean Absolute Error: 3.901570418326693
Mean Squared Error: 33.74061749653026
Root Mean Squared Error: 5.808667445854536
R2 Score: -0.00014313873003102273
Adjusted R2 Score: -0.3890876926805986


Save best model as Pickle

In [17]:
import pickle

# Save model
file_path = 'Saved Models/Lenigheid.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(best_model_mlr, file)

# Save features
with open('Saved Models/best_features_Lenigheid.txt', 'w') as file:
    file.write(f'{best_features_mlr}')